In [1]:
!pip install transformers
!pip install datasets

You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
# Load Dataset

from pathlib import Path
import re
from datasets import load_dataset

#data_files = {"train": "data/vvh_allg_bert_train.json", "test": "data/vvh_allg_bert_test.json"}
#data = load_dataset("json", data_files=data_files, field="data")

#import json

#with open("data/vvh_allg.json", mode="r", encoding="utf-8") as inf:
#    data = json.load(inf)
#labels = ["VVH-Allg", "Keine"]

# Reformat for use in BERT
#new_data = []
#for i in data:
#    if i["label"] == "VVH-Allg": l = 0
#    elif i["label"] == "Keine": l = 1
#    new_data.append({"text": i["text"], "label": l})

In [3]:
# Load test Dataset

# Reformat germeval for use in BERT:

import json
from datasets import load_dataset

# Already wrote the Json-files, no need to run again

"""
with open("germeval/germeval2018.test.txt", mode="r", encoding="utf-8") as infile:
    test = infile.readlines()
    
with open("germeval/germeval2018.training.txt", mode="r", encoding="utf-8") as inf:
    train = inf.readlines()
    
def assemble(data):
    new_data = []
    for i in data:
        j = i.split("\t")
        point = dict()
        point["text"] = j[:-2][0]
        if j[-2] == "OFFENSE":
            point["label"] = 1
        else: point["label"] = 0
        new_data.append(point)
    return new_data

testing = assemble(test)
training = assemble(train)   
    
train_data = {"data": training}
test_data = {"data": testing}

t_1 = json.dumps(train_data)
t_2 = json.dumps(test_data)

with open("germeval/germeval2018_train.json", mode="w", encoding="utf-8") as outfile:
    outfile.write(t_1)
    
with open("germeval/germeval2018_test.json", mode="w", encoding="utf-8") as outf:
    outf.write(t_2)"""


data_files = {"train": "data/vvh-tatkat_grob_bert_train.json", "test": "data/vvh-tatkat_grob_bert_test.json"}
data = load_dataset("json", data_files=data_files, field="data")

Using custom data configuration default-01a7a55fbbf24a37


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/json/default-01a7a55fbbf24a37/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Preprocess Dataset
from transformers import AutoTokenizer

# Tokenization und Padding
tokenizer = AutoTokenizer.from_pretrained(
    'bert-base-german-cased',
    padding=True,
    truncation=True,
    return_tensors="pt")

def preprocess(data):
    return tokenizer(data["text"])#, truncation=True, padding=True)

tokenized = data.map(preprocess, batched=True)
#tok = list(tokenized)

# Tags
#tags = [dpoint["label"] for dpoint in new_data]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [5]:
# Data-Collator
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
# Metrics
from datasets import load_metric
import numpy as np
metric = load_metric('matthews_correlation')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def compute_metrics_alt(eval_pred):
    predictions, labels = eval_pred
    #print(predictions, labels)
    
    predictions = np.argmax(predictions, axis=-1)    

    # Remove ignored index (special tokens)
    #true_predictions = [
    #    [p for (p, l) in zip(prediction, label) if l != -100]
    #    for prediction, label in zip(predictions, labels)
    #]
    #true_labels = [
    #    [l for (p, l) in zip(prediction, label) if l != -100]
    #    for prediction, label in zip(predictions, labels)
    #]
                
    #return metric.compute(average="weighted")
        
    metric = load_metric("f1")
    #for i in range(len(true_predictions)):
    #    metric.add_batch(predictions=true_predictions[i], references=true_labels[i])
    f1 = metric.compute(predictions=predictions, references=labels, average=None)
    
    metric = load_metric("precision")
    #for i in range(len(true_predictions)):
    #    metric.add_batch(predictions=true_predictions[i], references=true_labels[i])
    precision = metric.compute(predictions=predictions, references=labels, average=None)
    
    metric = load_metric("recall")
    #for i in range(len(true_predictions)):
    #    metric.add_batch(predictions=true_predictions[i], references=true_labels[i])
    recall = metric.compute(predictions=predictions, references=labels, average=None)
    
    #final_results = {}
    #for key, value in f1.items():
    #    prec = precision[key]
    #    rec = recall[key]
    #    final_results[key] = (prec, rec, value)
    final_results = {}
    final_results["f1"] = f1["f1"]
    final_results["precision"] = precision["precision"]
    final_results["recall"] = recall["recall"]
    
    return final_results


In [7]:
# Fine-Tuning
 
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy = "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_alt
)

trainer.train()

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,No log,0.452270,[0.13333333 0.89256198],[0.75 0.81],[0.07317073 0.99386503]
2,No log,0.454901,[0.39344262 0.89337176],[0.6 0.8423913],[0.29268293 0.95092025]
3,No log,0.705594,[0.32786885 0.88184438],[0.5 0.83152174],[0.24390244 0.93865031]
4,No log,0.882583,[0.37837838 0.86227545],[0.42424242 0.84210526],[0.34146341 0.88343558]
5,No log,1.046644,[0.36111111 0.86309524],[0.41935484 0.83815029],[0.31707317 0.88957055]
6,No log,1.151092,[0.32786885 0.88184438],[0.5 0.83152174],[0.24390244 0.93865031]
7,No log,1.235961,[0.34920635 0.88115942],[0.5 0.83516484],[0.26829268 0.93251534]
8,No log,1.358982,[0.37837838 0.86227545],[0.42424242 0.84210526],[0.34146341 0.88343558]
9,No log,1.312802,[0.38235294 0.87647059],[0.48148148 0.84180791],[0.31707317 0.91411043]


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 204
  Batch size = 16
Trainer is attempting to log a value of "[0.13333333 0.89256198]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.75 0.81]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.07317073 0.99386503]" of type <class 'numpy.ndarray'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribut

TypeError: Object of type ndarray is not JSON serializable